# Car-following model (Latent class basic specification)

This notebook was written by Evangelos Paschalidis (evangelos.paschalidis@epfl.ch) for the Decision-aid methodologies in transportation course at EPFL (http://edu.epfl.ch/coursebook/en/decision-aid-methodologies-in-transportation-CIVIL-557). 

Please contact before distributing or reusing the material below.

## Overview

This notebook covers the estimation of a GM car-following model in python with maximum likelihhod estimation:

* Load necessary packages
* Define variables and parameters to estimate
* Model specification
* Model output

Have a go at working through the notebook. To run a code cell, just click on it (to see a green box around it) and then press the **Run** button at the top! 

Some cells have blank lines for you to complete. There is always a comment telling you what to do!

You can also add a new cell by pressing the **+** button at the top of the page.

## Load packages

Before we estimate the model let's load some packages that we are going to need. When importing a package, it is common to rename it using an abbreviation.

In [1]:
import pandas as pd # for data frame processing
import numpy as np # for some statistical procedures
from scipy.optimize import minimize # optimisation routine for parameter estimation
from scipy.stats import norm # normal distribution density function
import numdifftools as nd # we use this to calculate t-ratios and p-values
import csv # we need this to store our parameters as csv

# We've got some rookies in the mix.
from scipy.special import roots_legendre # we use this to generate nodes for numerical integration
from scipy.stats import qmc # we use this to generate nodes for numerical integration

### Let's give a name to our model

In [2]:
model_name = 'car_following_model_latent_distance' # Name we want to give to our model (this is used when saving the output)

### Panel structure
We need to define whether our data is panel (i.e. multiple observations per individual) or not

In [3]:
panel = 1 # switch to 1 if data is panel (any other value if not panel)

### Define if we use mixing

In [4]:
mixing = 0 # switch to 1 if we apply mixing (any other value if no mixing applied)

## Load the data

Now it is time to load the data. We can do that using the piece of code below.

**Important!** Make sure the data is in the same folder with the notebook file

In [5]:
# Command to load the data
data = pd.read_table('Project_data/I80_data1.txt')

# Number of observations (we need this to caclulate goodness-of-fit indices)
Nobs = data.shape[0]

## Print the data

Let's quickly print the data. Simply type *data* in the field below

(If we had opened our data set with a different name e.g. *database*

In [6]:
# Type "data" in this field (without the quotation) and run the cell (Shift + return)
data

,ID,Time,Position,Length,Width,Type,Speed,Acceleration,Lane,Leader,...,lead04,lane01,lane02,lane03,lane04,tasks,task_index,timelag,dd,idd
0,11,23,69.326102,5.273601,-9999.99,2,5.110310,0.000229,2,1,...,1,1,1,1,1,65,5,22,1,11
1,11,24,74.436263,5.273601,-9999.99,2,5.110201,-0.000140,2,1,...,1,1,1,1,1,65,6,23,1,11
2,11,25,79.463722,5.273601,-9999.99,2,4.717329,-1.677461,2,1,...,1,1,1,1,1,65,7,24,1,11
3,11,26,83.030233,5.273601,-9999.99,2,3.076990,-0.227369,2,1,...,1,1,1,1,1,65,8,25,1,11
4,11,27,86.039234,5.273601,-9999.99,2,2.868391,-0.722239,2,1,...,1,1,1,1,1,65,9,26,1,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15337,3333,104,414.956622,4.663900,-9999.99,2,6.507050,-1.186400,6,103,...,1,1,1,1,1,51,47,103,1,3333
15338,3333,105,420.985633,4.663900,-9999.99,2,5.484839,-2.233160,6,103,...,1,1,1,1,1,51,48,104,1,3333
15339,3333,106,425.772685,4.663900,-9999.99,2,4.616979,-0.048320,6,103,...,1,1,1,1,1,51,49,105,1,3333
15340,3333,107,430.384415,4.663900,-9999.99,2,4.611069,-0.000101,6,103,...,1,1,1,1,1,51,50,106,1,3333


## Print the variable names (columns)

We can also print the variable names only using the piece of code below

* This is useful during model specification to easily access the column names

In [7]:
print(data.columns)

Index(['ID', 'Time', 'Position', 'Length', 'Width', 'Type', 'Speed',
       'Acceleration', 'Lane', 'Leader', 'Follower', 'Space_headway',
       'Time_headway', 'Acceleration_lead', 'Speed_lead', 'Position_lead',
       'Density', 'distanceToEnd', 'lane_change', 'lead_change', 'lag_leader0',
       'lag_leader1', 'lag_leader2', 'lag_leader3', 'lag_leader4', 'lag_lane0',
       'lag_lane1', 'lag_lane2', 'lag_lane3', 'lag_lane4', 'lag_acceleration0',
       'lag_acceleration1', 'lag_acceleration2', 'lag_acceleration3',
       'lag_acceleration4', 'lag_acceleration_lead0', 'lag_acceleration_lead1',
       'lag_acceleration_lead2', 'lag_acceleration_lead3',
       'lag_acceleration_lead4', 'lag_speed0', 'lag_speed1', 'lag_speed2',
       'lag_speed3', 'lag_speed4', 'lag_speed_lead0', 'lag_speed_lead1',
       'lag_speed_lead2', 'lag_speed_lead3', 'lag_speed_lead4',
       'lag_position_lead0', 'lag_position_lead1', 'lag_position_lead2',
       'lag_position_lead3', 'lag_position_lead4', '

## Data processing

* It may be the case that our data do not include all variables required for modelling.
* In this case we must generate the additional variables that we need

### Variable creation

* Let's create a "running task" variable, else a counter for the observations of each driver

In [8]:
data['running_task'] = data.groupby(['ID']).cumcount()+1 # counter of observations per individual

## Variable definition

We need to define the variables (as numpy arrays) that we will use in our model.

* The arrays can have any name but it is more convenient to use the same name as in the data set.

In [9]:
# Example:
# Variable_name = np.array(data['Variable_name']).reshape(-1, 1)

running_task = np.array(data['running_task']).reshape(-1, 1)
Acceleration = np.array(data['Acceleration']).reshape(-1, 1)
Speed = np.array(data['Speed']).reshape(-1, 1)
Space_headway = np.array(data['lag_s_headway1']).reshape(-1, 1)
Speed_lead = np.array(data['Speed_lead']).reshape(-1, 1)
Acceleration_lead = np.array(data['Acceleration_lead']).reshape(-1, 1)

#### Define the "lagged" speed variables

In [10]:
# Lagged speed values #
lag_speed1 = np.array(data['lag_speed1']).reshape(-1, 1)
lag_speed2 = np.array(data['lag_speed2']).reshape(-1, 1)
lag_speed3 = np.array(data['lag_speed3']).reshape(-1, 1)

# Lagged speed lead values #
lag_speed_lead1 = np.array(data['lag_speed_lead1']).reshape(-1, 1)
lag_speed_lead2 = np.array(data['lag_speed_lead2']).reshape(-1, 1)
lag_speed_lead3 = np.array(data['lag_speed_lead3']).reshape(-1, 1)

# Lagged acceleration values #
lag_acceleration1 = np.array(data['lag_acceleration1']).reshape(-1, 1)
lag_acceleration2 = np.array(data['lag_acceleration2']).reshape(-1, 1)
lag_acceleration3 = np.array(data['lag_acceleration3']).reshape(-1, 1)

# Lagged speed lead values #
lag_acceleration_lead1 = np.array(data['lag_acceleration_lead1']).reshape(-1, 1)
lag_acceleration_lead2 = np.array(data['lag_acceleration_lead2']).reshape(-1, 1)
lag_acceleration_lead3 = np.array(data['lag_acceleration_lead3']).reshape(-1, 1)

#### Define the ID variable

In [11]:
ID = np.array(data['ID']) # ID does not need to be reshaped

## Model specification

We now need to create a function that includes our model.

* Python functions are defined as: def *function_name*(parameters):
* We end a function as: return *value_to_return*

In the current implementation we specify two different functions as:
* *function 1* calculates the log likelihood per observations
* *function 2* calculates the sum of log likelihood taking as input the result from *function 1*

*We define two different functions to be more flexible in the post estimation processing later in the code*

We use some python (numpy) functions such '*exp*' or '*log*'. To execute these in the current example, we need to also call numpy; hence, we have *np.exp()* and *np.log()*.

### Define parameters and starting values

Ultimately, we want to estimate the value of some parameters that maximise the likelihood of our observations of the dependent variable.

Before starting the estimation process, we need to define some starting values for the parameters to be estimated.

* The starting values are usually zeroes
* When a parameter is included as a denominator, the starting value cannot be 0 for computational reasons.
* However, since we estimate the log of sigma, our starting value can be zero since exp(sigma) can never be absolute zero.

In [12]:
betas_start = {
               "bc_acc":0, # Constant of acceleration utility
               "bc_dec":0, # Constant of deceleration utility
                
                "b_speed_c1":0, # Relative speed for acceleration utility
                "b_speed_c2":0, # Relative speed for deceleration utility
                
               "alpha_acc":0, # Acceleration constant parameter
               "alpha_dec":0, # Deceleration constant parameter
               "beta_acc":0,  # Speed (acceleration regime) parameter
               "beta_dec":0,  # Speed (deceleration regime) parameter
                "gamma_acc":0, # Space headway (acceleration regime) parameter
                "gamma_dec":0, # Space headway (deceleration regime) parameter
               "lamda_acc_p":0, # Relative speed (acceleration regime) parameter
               # "lamda_acc_n":0, # Relative speed (deceleration regime) parameter
               # "lamda_dec_p":0, # Relative speed (acceleration regime) parameter
               "lamda_dec_n":0, # Relative speed (deceleration regime) parameter
                "dn":0,
               "sigma_acc":-0, # Std deviation (acceleration regime) parameter
               "sigma_dec":-0,  # Std deviation (deceleration regime) parameter
                "sigma_dn":-0  # Std deviation (deceleration regime) parameter
                }

### Load old parameter estimates results

Sometimes, we want to use results from old models as starting values.
* To do that, we will load the iteration file from a previous estimation
* Please note that only values of parameters with same names with our current model will be copied

In [13]:
### Activate this cell to load old results ###

# Open old iteration file
betas_old = pd.read_csv('car_following_model_latent_class_iterations.csv')

# Keep last row
betas_old = betas_old.tail(1)

# Convert to dictionary
betas_old = betas_old.to_dict(orient='records')[0]

# Copy values from old to start for keys that are common to both dictionaries
betas_start = {k: betas_old[k] for k in betas_start.keys() & betas_old.keys()}

# Delete old estimates
del[betas_old]

### Generation of random draws
In this piece of code below, we generate the random draws to be used as reaction time.

Two pieces of code are provided one for numerical integration and one for simulation

**Only one of the two blocks can be activated at a time!!**

In [14]:
# # First let's set the number of draws
# # This indicates the number of random draws per random effect
# Ndraws = 1

# # And then let's set the reaction time range
# tau_max = 3 # Max bound of reaction time
# tau_min = 0 # Min bound of reaction time

#### Method 1: Code for numerical integration
With numerical integration we directly get reaction time values in the (0, 3) range (or any other range we set)

In [15]:
# # This piece of code is generating the nodes and weights using the number of nodes we set just above
# roots, weights = roots_legendre(Ndraws) # generate (-1, 1) nodes

# tau_max = 3 # Max bound of reaction time
# tau_min = 0 # Min bound of reaction time
# tau = 0.5*(tau_max-tau_min)*roots+0.5*(tau_max+tau_min) # convert nodes (-1, 1) to reaction time values (0, 3)

# # Some data manipulation to correct the reaction time matix dimensions
# tau = np.transpose(tau)
# tau = np.tile(tau, (len(data), 1))

#### Method 2: Code for simulation
The simulation code will generate uniform random numbers (0, 1).

These numbers must be then used in the model function (Function 1) to convert them to reaction time values using truncated distribution simulation.

In [16]:
# # This piece of code is generating Halton draws using the number of nodes we set just above
# nIndiv = len(set(ID))
# draws = ['tau_draws']
# nDim = len(draws)
# tasks = (pd.DataFrame(ID).value_counts(sort = False)) # observations per ID

# sampler = qmc.Halton(d=nDim, scramble=False)
# sample = pd.DataFrame(sampler.random(n=Ndraws*nIndiv+Ndraws)) # +Ndraws
# sample = sample[(Ndraws-1):(Ndraws*nIndiv+Ndraws-1)]

# cols = len(sample.columns)

# for i in range(cols):
#     # print(i)
#     sample1 = np.array(sample.loc[:,i])
#     sample1=pd.DataFrame(np.reshape(sample1,(nIndiv,Ndraws)))
#     sample_rep =sample1.loc[sample1.index.repeat(tasks)]
#     globals()[draws[i]] = sample_rep

In [17]:
# Convert to normal distribution (or any other preferred)
# Here we use some funciton to convert the uniform draws we obtained
# in the previous block of code to any distirbution
# Example: draws_variable = norm.ppf(draws_variable)
# The norm.ppf() command converts the uniform draws to standard normal

# NOTE: This piece of code is not relevant process for the car-following model
# We do this distribution transformation process within Function 1

#### Calculate lagged speed values based on the reaction time values

This step is **Only** relevant for numerical integration.

If simulation is used then the block of code below must be pasted inside Function 1 and not before it.

In [18]:
# #################################################################################################
# # Here we calculate the lagged relative speed given the reaction time values we generated above #
# #################################################################################################

# # The way this is done is by implementing linear interpolation between the observations around each reaction time value
# # We simply assume that "acceleration = (speed1 - speed0) / (time1 - time0)"
# # We take the observation closest to the reaction time value

# # We calculate lag speed
# lag_speed = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed1+lag_acceleration1*(np.intc(tau)+1-tau))+
#                                     (np.intc(tau)+1-tau>=0.5)*(Speed-Acceleration*(tau-np.intc(tau))))+
#              (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed2+lag_acceleration2*(np.intc(tau)+1-tau))+
#                             (np.intc(tau)+1-tau>=0.5)*(lag_speed1-lag_acceleration1*(tau-np.intc(tau))))+
#              (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed3+lag_acceleration3*(np.intc(tau)+1-tau))+
#                             (np.intc(tau)+1-tau>=0.5)*(lag_speed2-lag_acceleration2*(tau-np.intc(tau)))))

# # We correct lag speed if the result is negative
# lag_speed = (lag_speed*(lag_speed>=0))+((np.round(tau)==0)*Speed+(np.round(tau)==1)*lag_speed1+(np.round(tau)==2)*lag_speed2+(np.round(tau)==3)*lag_speed3)*(lag_speed<0)

# # We calculate lag speed of leader
# lag_speed_lead = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead1+lag_acceleration_lead1*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(Speed_lead-Acceleration_lead*(tau-np.intc(tau))))+
#                   (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead2+lag_acceleration_lead2*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead1-lag_acceleration_lead1*(tau-np.intc(tau))))+
#                   (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead3+lag_acceleration_lead3*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead2-lag_acceleration_lead2*(tau-np.intc(tau)))))

# # We correct the lag speed of leader if the result is negative
# lag_speed_lead = (lag_speed_lead*(lag_speed_lead>=0))+((np.round(tau)==0)*Speed_lead+(np.round(tau)==1)*lag_speed_lead1+(np.round(tau)==2)*lag_speed_lead2+(np.round(tau)==3)*lag_speed_lead3)*(lag_speed_lead<0)


# # We calculate lagged relative speed as the difference
# Lagged_relative_speed = lag_speed_lead-lag_speed # Lagged relative speed per reaction time value

### Function 1: log likelihood (LL)
This function calculates the log likelihood per observation

In [19]:
def LL(betas): # betas is a vector with the parameters we want to estimate
   
    # First let's define the parameters to be estimated.
    # The parameter names are imported directly from 'beta_start' that we defined earlier
    
    for pn in range(len(betas_start.values())):
        globals()[np.array(list(betas_start.keys()))[pn]] = betas[pn]

    ############################################################################################################    
    ############################################################################################################    
    # Calculation of reaction time values
    # This part of the code must ONLY be activated if simulation is used
    # Below we convert the uniofrm numbers (tau_draws) to truncated log-normal
    
#     tau = np.exp(norm.ppf(
#        tau_draws*(norm.cdf(np.log(tau_max),tau_mean,np.exp(sigma_tau))) # -norm.cdf(np.log(tau_min),tau_mean,np.exp(sigma_tau))
#        )*np.exp(sigma_tau)+tau_mean)
    
    # #################################################################################################
    # # Here we calculate the lagged relative speed given the reaction time values we generated above #
    # #################################################################################################

    # # The way this is done is by implementing linear interpolation between the observations around each reaction time value
    # # We simply assume that "acceleration = (speed1 - speed0) / (time1 - time0)"
    # # We take the observation closest to the reaction time value
    
    # We calculate lag speed
#     lag_speed = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed1+lag_acceleration1*(np.intc(tau)+1-tau))+
#                                         (np.intc(tau)+1-tau>=0.5)*(Speed-Acceleration*(tau-np.intc(tau))))+
#                  (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed2+lag_acceleration2*(np.intc(tau)+1-tau))+
#                                 (np.intc(tau)+1-tau>=0.5)*(lag_speed1-lag_acceleration1*(tau-np.intc(tau))))+
#                  (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed3+lag_acceleration3*(np.intc(tau)+1-tau))+
#                                 (np.intc(tau)+1-tau>=0.5)*(lag_speed2-lag_acceleration2*(tau-np.intc(tau)))))

#     # We correct lag speed if the result is negative
#     lag_speed = (lag_speed*(lag_speed>=0))+((np.round(tau)==0)*Speed+(np.round(tau)==1)*lag_speed1+(np.round(tau)==2)*lag_speed2+(np.round(tau)==3)*lag_speed3)*(lag_speed<0)

#     # We calculate lag speed of leader
#     lag_speed_lead = ((np.intc(tau)==0) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead1+lag_acceleration_lead1*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(Speed_lead-Acceleration_lead*(tau-np.intc(tau))))+
#                       (np.intc(tau)==1) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead2+lag_acceleration_lead2*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead1-lag_acceleration_lead1*(tau-np.intc(tau))))+
#                       (np.intc(tau)==2) * ((np.intc(tau)+1-tau<0.5)*(lag_speed_lead3+lag_acceleration_lead3*(np.intc(tau)+1-tau))+(np.intc(tau)+1-tau>=0.5)*(lag_speed_lead2-lag_acceleration_lead2*(tau-np.intc(tau)))))

#     # We correct the lag speed of leader if the result is negative
#     lag_speed_lead = (lag_speed_lead*(lag_speed_lead>=0))+((np.round(tau)==0)*Speed_lead+(np.round(tau)==1)*lag_speed_lead1+(np.round(tau)==2)*lag_speed_lead2+(np.round(tau)==3)*lag_speed_lead3)*(lag_speed_lead<0)


#     # We calculate lagged relative speed as the difference
#     Lagged_relative_speed = lag_speed_lead-lag_speed # Lagged relative speed per reaction time value
    
    Lagged_relative_speed = lag_speed_lead1-lag_speed1
    # Lagged_relative_speed = Speed_lead-Speed
    
    
    ############################################################################################################
    ############################################################################################################
    
    # Latent classes
    
    # Utility functions
    U_acc = bc_acc + b_speed_c1*Lagged_relative_speed*(Lagged_relative_speed>0)
    U_dec = bc_dec + b_speed_c2*Lagged_relative_speed*(Lagged_relative_speed<0)
    U_dn = 0

    # Utility exponentials
    U_acc = np.exp(U_acc)
    U_dec = np.exp(U_dec)
    U_dn = np.exp(U_dn)
    
    # Sum of utilities
    sum_all = U_acc + U_dec + U_dn
    
    # Probabilities
    P_acc = U_acc/sum_all
    P_dec = U_dec/sum_all
    P_dn = U_dn/sum_all
    
    ############################################################################################################
    ############################################################################################################
    
    # Car-following model specification

    # BE CAREFUL!! Below we add a small correction term (np.exp(-50)) to speed and relative speed.
    # This correction is included to avoid estimation issues if the value of the independent variables is 0.
    
    # BE CAREFUL!! We force the signs of alpha_acc and alpha_dec by using then inside an exp() funciton.
    
    # Sensitivity term
    sensitivity_acc = np.exp(alpha_acc)*((Speed+np.exp(-50))**beta_acc)/(Space_headway**gamma_acc)
    sensitivity_dec = -np.exp(alpha_dec)*((Speed+np.exp(-50))**beta_dec)/(Space_headway**gamma_dec)

    # Stimulus term (we add a condition to keep only the positive values for acceleration and negative for deceleration)
    stimulus_acc = (
        (np.abs(Lagged_relative_speed + np.exp(-50))**lamda_acc_p)**(Lagged_relative_speed>=0)
        # (np.abs(Lagged_relative_speed + np.exp(-50))**lamda_acc_n)*(Lagged_relative_speed<0)
        )
    
    stimulus_dec = (
        # (np.abs(Lagged_relative_speed + np.exp(-50))**lamda_dec_p)*(Lagged_relative_speed>=0)+
        (np.abs(Lagged_relative_speed + np.exp(-50))**lamda_dec_n)**(Lagged_relative_speed<0)
        )

    # Acceleration - deceleration means
    acc = sensitivity_acc*stimulus_acc
    dec = sensitivity_dec*stimulus_dec

    # Density functions for acceleration and deceleration
    pdf_acc = norm.pdf(Acceleration,acc,np.exp(sigma_acc))/(1-norm.cdf(0,acc,np.exp(sigma_acc)))
    pdf_dec = norm.pdf(Acceleration,dec,np.exp(sigma_dec))/(norm.cdf(0,dec,np.exp(sigma_dec)))
    
    # Density function for do-nothing acceleration
    pdf_dn_pos = norm.pdf(Acceleration, dn, np.exp(sigma_dn)) / (1 - norm.cdf(0, dn, np.exp(sigma_dn)))

    # Density function for do-nothing deceleration
    pdf_dn_neg = norm.pdf(Acceleration, dn, np.exp(sigma_dn)) / norm.cdf(0, dn, np.exp(sigma_dn))




    P_acc_dn = 1 - norm.cdf(0, dn, np.exp(sigma_dn))

    P_dec_dn = norm.cdf(0, dn, np.exp(sigma_dn))

    
    # Combined probability of acceleration and deceleration regimes

    f_nt_acc = (pdf_acc * (pdf_acc / (pdf_acc + P_acc_dn * P_dn)) + pdf_dn_pos * (P_acc_dn * P_dn / (pdf_acc + P_acc_dn * P_dn)))
    f_nt_dec = (pdf_dec * (P_dec / (P_dec + P_dec_dn * P_dn)) + pdf_dn_neg * (P_dec_dn * P_dn / (P_dec + P_dec_dn * P_dn)))


    P = f_nt_acc * (Acceleration >= 0) + f_nt_dec * (Acceleration < 0)

    
    
    ############################################################################################################
    # - Now this below is relevant if we have panel data and apply mixing (Do not change this piece of code!) -#
    if panel == 1:
    # Do it as panel
        P = pd.DataFrame(P)
        P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
        P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
        # We take the product of probabilities per individual per draw and then delete the ID column
        P = P.groupby('ID', as_index=False).prod()
        P = P.drop('ID', axis=1)
   
    if mixing == 1:
        # We take the average per row to get the average probability per individual (if mixing == 1)
        
        if panel == 1:
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
        
        if panel == 0:
            P = pd.DataFrame(P)
            P = pd.concat([pd.Series(ID), pd.DataFrame(P)], axis=1, ignore_index=True)
            P.rename(columns={P.columns[0]: 'ID'},inplace=True)
    
            # We take the product of probabilities per individual per draw and then delete the ID column
            P = P.groupby('ID', as_index=False).prod()
            P = P.drop('ID', axis=1)
            P['mean'] = P.mean(axis=1)
            P = np.array(P['mean'])
            
    P = np.array(P)
    ### --- This is where the panel data approach ends. --- ###
    ############################################################################################################
    
    # We then take the log of the density function
    logprob = np.log(P)
    
    return logprob

### Function 2: sum of log likelihood (SLL)
This function simply takes the sum of log likelihood that we calculated with the first function

In [20]:
def SLL(betas):
    return -sum(LL(betas))

## Model estimation

### Warnings

Sometimes, optimisation procedures may 'overdo' it with warnings during estimation.
We can supress these with the piece of code below (not always advisable)

In [21]:
import warnings
from scipy.stats import t
# with warnings.catch_warnings():
warnings.filterwarnings("ignore")

### Estimation

Now it is finally time to run our estimation command.
We use an optimisation algorith called 'BFGS'.

**It is advisable not to edit the lines of code below**

In [22]:
# This will give us the initial loglikelihood value as an output
def callback1(betas):
    print("Current log likelihood:", -SLL(betas))

# This function will allow as to store parameter estimates during iterations
# Initialise list to store parameter values
parameter_values = [np.array(list(betas_start.values()))]
# Then define the function
def callback2(betas):    
    parameter_values.append(betas)
    column_names = list(betas_start.keys())
    with open(f'{model_name}_iterations.csv','w',newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(column_names)
        writer.writerows(parameter_values)

# Now let's combine the two callback functions
def combined_callback(betas):
    callback1(betas)
    callback2(betas)
        
print("Initial log likelihood:", -SLL(np.array(list(betas_start.values()))))

# Choose optimisation routine (preferably BFGS)
optimiser = 'BFGS' # BFGS or L-BFGS-B or nelder-mead

result = minimize(SLL, np.array(list(betas_start.values())), method=optimiser,callback=combined_callback, 
                  options={'disp':False}) # ,bounds=bounds1
#args = (parameter_values,)
print("Final log likelihood:", -result.fun)

Initial log likelihood: [-5011.13366512]
Current log likelihood: [-5010.86563174]
Current log likelihood: [-5008.64778155]
Current log likelihood: [-5007.78900835]
Current log likelihood: [-5006.64736456]
Current log likelihood: [-5005.8905312]
Current log likelihood: [-5005.77549446]
Current log likelihood: [-5005.64683209]
Current log likelihood: [-5005.5274481]
Current log likelihood: [-5005.49631578]
Current log likelihood: [-5005.24886027]
Current log likelihood: [-5004.90525673]
Current log likelihood: [-5004.81480413]
Current log likelihood: [-5004.77057029]
Current log likelihood: [-5004.68575281]
Current log likelihood: [-5004.54171459]
Current log likelihood: [-5004.34453799]
Current log likelihood: [-5003.99137653]
Current log likelihood: [-5003.49079621]
Current log likelihood: [-5003.39395101]
Current log likelihood: [-5003.39380818]
Current log likelihood: [-5003.39380641]
Current log likelihood: [-5003.39380637]
Current log likelihood: [-5003.39380637]
Current log likeli

## Post estimation processing

We evaluate our parameter estimates using t-ratios (or p-Values).

In maximum likelihood estimation, we extract these from the variance-covariance matrix of the parameters.

The variance covariance matrix is not readily available but we need to calculate it.

This is done with the code below.

**DO NOT EDIT THE CHUNK OF CODE BELOW!**

In [23]:
# Vector of parameter estimates
parameters = result['x'] 

# Calculate hessian
print("Calculating Hessian, please wait (this may take a while...)")
Hess = nd.Hessian(SLL,method='forward')
hessian = Hess(parameters)
inv_hessian = np.linalg.inv(hessian)

# Parameter statistics
dof = Nobs - len(betas_start) - 1
se = np.sqrt(np.diag(inv_hessian)) # Standard errors
tratio = parameters/se # t-ratios
p_value = (1-t.cdf(np.abs(tratio),dof)) * 2 # p-values


# --- Sandwich estimator --- #

# The sandwich estimator provides the "robust" s.e., t-ratios and p-values.
# These should be preferred over the classical parameter statistics.

# We first need the gradients at the solution point
Grad = nd.Gradient(LL,method='forward')
gradient = Grad(parameters)

# Then we need to calculate the B matrix
B = np.array([])
for r in range(gradient.shape[0]):
    Bm = np.zeros([len(betas_start),len(betas_start)])
    gradient0 = gradient[r,:]
    for i in range(len(gradient0)):
            for j in range(len(gradient0)):
                element = gradient0[i]*gradient0[j]
                Bm[i][j] = element
    if B.size==0:
                    B = Bm
    else:
                    B = B+Bm

# Finally we "sandwich" the B matrix between the inverese hessian matrices
BHHH = (inv_hessian)@(B)@(inv_hessian)

print("... Done!!")

# Now it is time to calculate some "robust" parameter statistics
rob_se = np.sqrt(np.diag(BHHH)) # robust standard error
rob_tratio = parameters/rob_se # robust t-ratio
rob_p_value = (1-t.cdf(np.abs(rob_tratio),dof)) * 2 # robust p-value

Calculating Hessian, please wait (this may take a while...)
... Done!!


## Results
Finally, we got our results. Let's print them!

The outputs that we receive are:
* Estimates: These are the values of our parameters. We must check if the sign is consistent with our expectations
* s.e.: Standard errors of the parameters
* tratio: t-ratio of the parameters (significant if absolute value > 1.96)
* p_value: p-value of the parameters (significant if < 0.05)

The parameter statistics also have their **robust** versions. These should be preferred as they are less susceptible to mis-specified models.

In [24]:
arrays = np.column_stack((np.array(list(betas_start.keys())),parameters,se,tratio,p_value,rob_se,rob_tratio,rob_p_value))
results = pd.DataFrame(arrays, columns = ['Parameter','Estimate','s.e.','t-ratio0','p-value',
                                          'Rob s.e.','Rob t-ratio0','Rob p-value'])

results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']] = (
results[['Estimate','s.e.','t-ratio0','p-value','Rob s.e.','Rob t-ratio0','Rob p-value']].apply(pd.to_numeric, errors='coerce'))
numeric_cols = results.select_dtypes(include='number').columns
results[numeric_cols] = results[numeric_cols].round(3)
results # print results

,Parameter,Estimate,s.e.,t-ratio0,p-value,Rob s.e.,Rob t-ratio0,Rob p-value
0,gamma_dec,1.344,0.199,6.759,0.000,0.170,7.905,0.000
1,bc_acc,0.956,0.102,9.332,0.000,0.079,12.168,0.000
2,b_speed_c1,0.691,0.065,10.560,0.000,0.061,11.334,0.000
3,beta_dec,0.291,0.128,2.272,0.023,0.121,2.401,0.016
4,alpha_acc,0.384,0.030,12.988,0.000,0.046,8.299,0.000
5,sigma_dec,0.078,0.018,4.324,0.000,0.019,3.986,0.000
6,lamda_acc_p,0.043,0.005,8.565,0.000,0.007,5.737,0.000
7,alpha_dec,1.410,0.350,4.025,0.000,0.350,4.032,0.000
8,lamda_dec_n,1.709,0.183,9.315,0.000,0.181,9.433,0.000
9,beta_acc,0.019,0.012,1.612,0.107,0.018,1.098,0.272


## Goodness-of-fit indices

Let's calculate some goodness-of-fit indices now (do not edit)

In [25]:
# First let's calculate the GoF indices

rho_squared = 1 - ((-result.fun)/(-SLL(np.zeros(len(betas_start)))))
adj_rho_squared = 1 - (((-result.fun)-len(betas_start))/(-SLL(np.zeros(len(betas_start)))))

AIC = 2*len(betas_start) - 2*(-result.fun)
BIC = len(betas_start)*np.log(Nobs) - 2*(-result.fun)

LL0t = "Log likelihood at zeros:" + str(-SLL(np.zeros(len(betas_start))))
LLinit = "Initial log likelihood:" + str(-SLL(np.array(list(betas_start.values()))))
LLfin = "Final log likelihood:" + str(-result.fun)

rs1 = "rho squared="+str(rho_squared)
rs2 = "adjusted rho squared="+str(adj_rho_squared)
ac = "AIC="+str(AIC)
bc = "BIC="+str(BIC)

# Then let's print the GoF indices

print(LL0t)
print(LLinit)
print(LLfin)

print(rs1)
print(rs2)
print(ac)
print(bc)

Log likelihood at zeros:[-12961.23386546]
Initial log likelihood:[-5011.13366512]
Final log likelihood:-5003.393806371316
rho squared=[0.61397242]
adjusted rho squared=[0.61273796]
AIC=10038.787612742632
BIC=10161.00120385494


## Save output

We can save our output using the code below (do not edit):

In [26]:
with open(f"{model_name}_results.txt", 'w') as f:
    f.write(f'{LL0t}\n')
    f.write(f'{LLinit}\n')
    f.write(f'{LLfin}\n')
    f.write(f'{rs1}\n')
    f.write(f'{rs2}\n')
    f.write(f'{ac}\n')
    f.write(f'{bc}\n')
    results.to_csv(f, index=False, sep='\t')
results.to_csv(f'{model_name}_results.csv', index=False)